In [25]:
#Run any one cell to select a certain url

In [ ]:
#If facing issues with the default firefox driver for webscraping using selenium , run the below code
!sudo apt-get install chromium-chromedriver
!sudo apt-get update
!sudo apt-get install chromium-chromedriver
#Here the chrome driver is being used , hence replace firefox with Chrome driver as given below
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome(options=options)

In [26]:
#Alphabetical parse url
url = "https://www.espncricinfo.com/ci/content/player/country.html?country=6;alpha=A"

In [27]:
#Country based parse url
url = "https://www.espncricinfo.com/ci/content/player/index.html?country=6"

In [28]:
from bs4 import BeautifulSoup as bf
import requests
import pandas as pd
import time
from selenium import webdriver

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
driver = webdriver.Firefox()

In [29]:
req = requests.get(url)

In [30]:
soup = bf(req.text,"html.parser")

In [31]:
search_tag_container = soup.find_all('td')

In [32]:
#Fetch total urls of players belonging to a country
u = list()
for job_elem in search_tag_container:
    u.append(job_elem.find('a'))
    #result.append(job_elem.text)
l = set(u)
l = list(l)

In [33]:
def evaluate(html):
  select_divs = html.find_all('div',class_="card overflow-hidden mb-3")
  exist = 0
  location = 0
  for i in range(len(select_divs)):
    check = ""
    try:
      check = select_divs[i].find('p',class_="benton-bold pl-3 pt-4 pb-3 m-0 player-card-header").text
    except:
      k = 1
    if(check == "Career Averages"):
      exist = 1
      location = i
      break
  return exist,select_divs[location]

In [34]:
# Handling images
def get_player_image(player_url):
    # For fixing installation errors on linux: `sudo apt install firefox-geckodriver`
    global driver
    driver.get(player_url)
    time.sleep(2)
    images = driver.find_elements_by_tag_name('img')
    images = [image for image in images if "player-card__face" in image.get_attribute("class")]
    print(images[0].get_attribute("src"))
    return ["image"], [images[0].get_attribute("src")]

In [35]:
# Get trophy info
def get_major_trophies_information(major_trophies_soup):
    major_trophies_results = {}
    all_h5_tags = major_trophies_soup.find_all('h5', class_="border-bottom-gray-300 m-0 pl-3 pb-2 table-header")
    if all_h5_tags is None:
        return None
    required_h5_tag = all_h5_tags[0]
    for h5_tag in all_h5_tags:
        if str(h5_tag.text).strip() == "In Major Trophies":
            required_h5_tag = h5_tag
            break
    major_trophies_table_head = required_h5_tag.findNext('thead')
    major_trophies_table_headings = major_trophies_table_head.find_all('th')
    major_trophies_table_headings_count = 0
    major_trophies_headings_list = []
    for table_heading in major_trophies_table_headings:
        major_trophies_headings_list.append(table_heading.text)
    major_trophies_table_headings_count = len(major_trophies_headings_list)
    # print('Major Trophies table Head count', major_trophies_table_headings_count)
    current_title = "HOME"
    major_trophies_table_body = major_trophies_table_head.findNext('tbody')
    major_trophies_table_data = major_trophies_table_body.find_all('td')
    t = 0
    for table_data in major_trophies_table_data:
        attribute_number = t % major_trophies_table_headings_count
        if(attribute_number != 0):
            major_trophies_results[current_title][major_trophies_headings_list[attribute_number]] = table_data.text
        else:
            current_title = table_data.text
            major_trophies_results[current_title] = {}
        t += 1
    print(major_trophies_results)
    return ["Major trophies"], [major_trophies_results]

In [36]:
# Statistical analysis - Home, Away stats and function call to trophy info
def get_statistical_analysis_and_trophy_info(req):
    analysis_labels = []
    analysis_results = []
    analysis_url = req.url + "/bowling-batting-stats"
    #analysis_url = "https://www.espncricinfo.com/player/virat-kohli-253802/bowling-batting-stats"
    
    # selecting dropdown option
    global driver
    driver.get(analysis_url)
    # loading page
    time.sleep(2)
    dropdowns = driver.find_elements_by_class_name('dropdown-container')
    for dropdown in dropdowns:
        is_format_dropdown = str(dropdown.find_elements_by_tag_name('button')[0].text).strip() in ["Test", "ODI", "T20I", "Test+ODI+T20I"]
        is_role_dropdown = str(dropdown.find_elements_by_tag_name('button')[0].text).strip() in ["Batting", "Bowling", "Fielding", "Allround"]
        if (not is_format_dropdown) and (not is_role_dropdown):
            continue
        required_span_text = "Test+ODI+T20I"
        if is_role_dropdown:
            required_span_text = "Allround"
        dropdown.find_elements_by_tag_name('button')[0].click()
        all_list_items = dropdown.find_elements_by_tag_name('li')
        for list_item in all_list_items:
            span_text = list_item.find_elements_by_tag_name('span')[0].text
            if span_text == required_span_text:
                list_item.click()
                break
    # waiting for changes to load
    time.sleep(2)
    analysis_page = driver.page_source
    analysis_soup = bf(analysis_page, "html.parser")
    
    all_h5_tags = analysis_soup.find_all('h5', class_="border-bottom-gray-300 m-0 pl-3 pb-2 table-header")
    if all_h5_tags is None:
        return None
    required_h5_tag = all_h5_tags[0]
    for h5_tag in all_h5_tags:
        if str(h5_tag.text).strip() == "Home vs Away":
            required_h5_tag = h5_tag
            break
    analysis_table_head = required_h5_tag.findNext('thead')
    analysis_table_headings = analysis_table_head.find_all('th')
    analysis_table_headings_count = 0
    analysis_table_headings_list = []
    for table_heading in analysis_table_headings:
        analysis_table_headings_list.append(table_heading.text)
    analysis_table_headings_count = len(analysis_table_headings_list)
    # print('Analysis table Head count', analysis_table_headings_count)
    current_title = "HOME"
    analysis_table_body = analysis_table_head.findNext('tbody')
    analysis_table_data = analysis_table_body.find_all('td')
    t = 0
    for table_data in analysis_table_data:
        attribute_number = t % analysis_table_headings_count
        if(attribute_number != 0):
            analysis_results.append(table_data.text)
            analysis_labels.append(current_title + "_" + analysis_table_headings_list[attribute_number])
        else:
            current_title = table_data.text.upper()
        t += 1
    # for i in range(len(analysis_labels)):
    #     print(f'{analysis_labels[i]} : {analysis_results[i]}')
    trophy_label, trophy_result = get_major_trophies_information(analysis_soup)
    return analysis_labels + trophy_label, analysis_results + trophy_result

In [37]:
def find_relations(Relations):
    relations_container = Relations.find_next('div')
    individual_relations = relations_container.find_all('span')
    total_relations = list()
    for i in range(len(individual_relations)):
        name = individual_relations[i].find('h5').text
        bond = individual_relations[i].find('p').text
        total_relations.append([name,bond])
    return total_relations

In [38]:
#Get relevant information related to a player using each player's url
def load(req,final_list_players,ind,u,debut_url):
    lab = list()
    result = list()
    soup = bf(req.text,"html.parser")
    print(soup.title)
    search_tag_container = soup.find_all('h5',class_='player-card-description gray-900')
    count = 0
    for job_elem in search_tag_container:
        result.append(job_elem.text)
        count+=1
    search_tag_container = soup.find_all('p',class_="text-uppercase gray-700 mb-0 pb-0-5 player-card-heading")
    for job_elem in search_tag_container:
        lab.append(job_elem.text)
    ##################################
    Relations = soup.find('p',class_="text-uppercase gray-700 mb-0 player-card-heading text-uppercase")
    #relation_players = soup.find('p',class_="player-card-description player-description-link gray-900 m-0 player-link")
    if(Relations != None):
        if(Relations.text.lower()=='relations' or Relations.text.lower()=='relation'):
            lab.append("Relations")
            all_relations = find_relations(Relations)
            result.append([all_relations])
    ######################################
    rec = extra(u)
    lab.append("Records")
    result.append([rec])
    print(result)
    ##########################################
    records_format , records_data = format_records(u)
    for iter in range(len(records_format)):
      lab.append(records_format[iter])
      result.append([records_data[iter]])
    ###########################################
    matches = debut(debut_url)
    result.append([matches])
    lab.append("Matches")
    ##########################################
    lab.append("Teams")
    team_data = teams_data(soup)
    result.append([team_data])
    ###########################################
    analysis_and_trophy_labels, analysis_and_trophy_results = get_statistical_analysis_and_trophy_info(req)
    lab = lab + analysis_and_trophy_labels
    result = result + analysis_and_trophy_results
    ###########################################
    player_image_label, player_image_result = get_player_image(req.url)
    lab = lab + player_image_label
    result = result + player_image_result
    ###########################################
    career_span_tag = soup.find('div', class_="intl_career-desktop")
    career_span_tag_text = career_span_tag.findNext('p').text
    career_span_info = ""
    start_index = 0
    for i in range(len(career_span_tag_text)):
        if career_span_tag_text[i] == '1' or career_span_tag_text[i] == '2':
            start_index = i
            break
    career_span_tag_text = career_span_tag_text[start_index:]
    result.append(career_span_tag_text)
    lab.append("career_span")
    ###########################################
    search_tag_container = soup.find('div',class_="player-card__details")
    l = list()
    for job_elem in search_tag_container:
        l.append(job_elem.text)
        count+=1
    p = l[1].split("|")
    l.pop()
    l.pop()
    l = l+p
    result = result + l
    l_label = ["Player Name" , "Nationality ", "player-type"]
    lab = lab + l_label
    ##########################
    valid,soup = evaluate(soup)
    if(valid == 1):
      search_tag_container = soup.findAll('thead')
      if(len(search_tag_container)!=0):
        tag_content = search_tag_container[0].find_all('th')
        head = 0
        table_labels = list()
        table_labels_1 =list()
        for job_elem in tag_content:
            table_labels.append(job_elem.text)
            head+=1
        print('Head count',head)
        
        search_tag_container = soup.findAll('tbody')
        tag_content = search_tag_container[0].find_all('td')
        track_head_label = 0
        for job_elem in tag_content:
            if(track_head_label%head !=0):
                result.append(job_elem.text)
                count+=1
            else:
                table_labels_1.append(job_elem.text)
            
            track_head_label+=1
        total_label1 = list()
        for i in range(1,len(table_labels)):
            for j in range(len(table_labels_1)):
                total_label1.append('Batting' + table_labels[i] +" "+table_labels_1[j])
        lab = lab + total_label1
      ####################################
      search_tag_container = soup.findAll('thead')
      if(len(search_tag_container)>1):
        tag_content = search_tag_container[1].find_all('th')
        head = 0
        table_labels = list()
        table_labels_1 =list()
        for job_elem in tag_content:
            table_labels.append(job_elem.text)
            head+=1
        print('Head count',head)
        
        search_tag_container = soup.findAll('tbody')
        tag_content = search_tag_container[1].find_all('td')
        track_head_label = 0
        for job_elem in tag_content:
            if(track_head_label % head !=0):
                result.append(job_elem.text)
                count+=1
            else:
                table_labels_1.append(job_elem.text)
            
            track_head_label+=1
        total_label1 = list()
        for i in range(1,len(table_labels)):
            for j in range(len(table_labels_1)):
                total_label1.append('Bowling' + table_labels[i] +" "+table_labels_1[j])
        lab = lab + total_label1
    print('label--',len(lab))
    print('result--',len(result))
    
    lo = dict(zip(lab,result))
    lp = pd.DataFrame(lo,index=[ind])
    final_list_players.append(lp)
    print('count',count)

In [39]:
#Get individual player's "Records" information.
import re
def extra(l):
    re = requests.get(l)
    time.sleep(2)
    so = bf(re.text,"html.parser")
    res1 = so.find_all('div',class_='col-14 d-flex flex-row')
    res = so.find_all('div',class_='row bg-gray-100 class-record-row-p d-flex align-items-center')
    #print(res1)
    url_list = list()
    for i in range(len(res1)):

        pre = res1[i].find_all('h1')
        loi = res1[i].find_all('p',class_='record-class-title m-0')
        #print(loi)    
        for j in range(len(pre)):
            #print(pre[j].text)
            #print(loi[j].text)
            url_list.append(str(pre[j].text)  +str(" ") + str(loi[j].text))
    return url_list


In [40]:
def format_records(url):
  req = requests.get(url)
  soup = bf(req.text,"html.parser")
  record_sections = soup.find_all('div',class_="card content-block")
  required = ['Test Records','ODI Records','T20I Records']
  available_labels = list()
  labels_data = list()
  count = -1
  for i in range(len(record_sections)):
    name = record_sections[i].find('h5',class_="m-0")
    if(name.text in required):
      available_labels.append(name.text)
      labels_data.append([])
      count +=1
      individual_block_record = record_sections[i].find_all('a',class_="gray-900 benton-normal d-block")
      for i in range(len(individual_block_record)):
        try:
          a = individual_block_record[i].find('span',class_="m-0 benton-bold gray-1000 pr-3").text
          b = individual_block_record[i].find('span',class_="list-record-title").text
          c = individual_block_record[i].find('span',class_="m-0 ml-1 gray-600").text
          labels_data[count].append(str(a)+str(" ")+str(b)+str(" ")+str(c))
        except:
          no = 1
  return available_labels , labels_data

In [41]:
def teams_data(so):
  teams = []
  loi = so.find_all('h5',class_='m-0 ml-2 link-border-bottom player-description-link')
  for i in range(len(loi)):
    teams.append(loi[i].text)
  return teams

In [42]:
def debut(link):
  player_match = list()
  url = link
  request = requests.get(url)
  soup = bf(request.text,"html.parser")
  search_tag_container = soup.find_all('div',{'id':'debut-last-matches'})
  cp_jo = search_tag_container
  if(len(search_tag_container) != 0):
    total = cp_jo[0].find_all('div',class_="")
    for i in range(len(total)):
      p =total[i]
      a= p.find('h5',{'class':"benton-bold player-matches-subtitle d-flex justify-content-between align-items-center player_matches-dropdown m-0"})
      k = total[i].find_all('span',class_="match-event-row-debut-last white-space-nowrap")
      search_tag_container = total[i].find_all('span',{"class": "player-match-link"})
      new = total[i].find_all('div',{"class":"col-12 player-match-event-cell white-space-nowrap"})
      #print(a)
      #print(k[0])
      #print(new)
      try:
        l =a.span.decompose()
      except:
        player_match.append(a.text)
        #print(a.text)
      else:
        player_match.append(a.text)
        #print(a.text)
      for j in range(len(k)):
        try:
          temp = search_tag_container[j].text
        except:
          temp = new[j].text
        append = k[j].text + str(" ") + temp
        player_match.append(append)
        #print(k[j].text,end=" ")
        #print(search_tag_container[j].text)
  return player_match

In [43]:
import time
final_list_players = list()
for i in range(10):
    u = "https://www.espncricinfo.com"
    rl = l[i]["href"]
    link = u+rl
    print(link)
    #link = "https://www.espncricinfo.com/player/virat-kohli-253802"
    req = requests.get(link)
    time.sleep(2)
    print('player-',i,end=" ")
    ind = i
    record_url = req.url+str("/tests-odi-t20-records")
    debut_url = req.url+str("/matches")
    #debut(debut_url)
    load(req,final_list_players,ind,record_url,debut_url)
    #print(record_url)
    #jk = extra(record_url)
    #print(jk)
driver.close()

https://www.espncricinfo.com/ci/content/player/481896.html
player- 0 <title>Mohammed Shami profile and biography, stats, records, averages, photos and videos</title>
['Mohammed Shami Ahmed', 'September 03, 1990, Amroha, Uttar Pradesh', '30y 267d', 'Right hand bat', 'Right arm fast', 'Bowler', [[['Mohammed Kaif', ' (brother)']]], [['10th Best career strike rate (27.3)', '1st Most consecutive four-wickets-in-an-innings (3)', '7th Fastest to 100 wickets (56)']]]
{'World Cup': {'Span': '2015-2019', 'Mat': '11', 'Runs': '10', 'HS': '4*', 'Bat Avg': '3.33', '100s': '0', 'Wkts': '31', 'BBI': '5/69', 'Bowl Avg': '15.70', '5w': '1', 'Ct': '8', 'St': '0', 'Avg Diff': '-12.37'}, 'Asia Cup': {'Span': '2014-2014', 'Mat': '4', 'Runs': '14', 'HS': '14*', 'Bat Avg': '14.00', '100s': '0', 'Wkts': '9', 'BBI': '4/50', 'Bowl Avg': '25.55', '5w': '0', 'Ct': '1', 'St': '0', 'Avg Diff': '-11.55'}, 'Border-Gavaskar': {'Span': '2014-2020', 'Mat': '8', 'Runs': '74', 'HS': '34', 'Bat Avg': '10.57', '100s': '0', 

https://img1.hscicdn.com/image/upload/f_auto,t_gn_f_345/lsci/db/PICTURES/CMS/316500/316521.png
Head count 14
Head count 15
label-- 219
result-- 219
count 171
https://www.espncricinfo.com/ci/content/player/422108.html
player- 7 <title>KL Rahul profile and biography, stats, records, averages, photos and videos</title>
['Kannaur Lokesh Rahul', 'April 18, 1992, Bangalore, Karnataka', '29y 40d', 'Right hand bat', 'Wicketkeeper', 'Opening batter', [['1st Fifties in consecutive innings (7)', '2nd Most catches in a series (14)', ' Hundred on debut (100*)', '2nd Most runs in an innings (by batting position) (110*)', '7th Highest career batting average (39.92)', '3rd Fifties in consecutive innings (3)', '4th Fastest to 1000 runs (29)', '2nd Fifties in consecutive innings (7)']]]
{'World Cup': {'Span': '2019-2019', 'Mat': '9', 'Runs': '361', 'HS': '111', 'Bat Avg': '45.12', '100s': '1', 'Ct': '1', 'St': '0'}, 'Asia Cup': {'Span': '2018-2018', 'Mat': '1', 'Runs': '60', 'HS': '60', 'Bat Avg': '60.0

In [44]:
a = final_list_players[0]

In [45]:
a

,Full Name,Born,Age,Batting Style,Bowling Style,Playing Role,Relations,Records,Test Records,ODI Records,T20I Records,Matches,Teams,HOME_Span,HOME_Mat,HOME_Runs,HOME_HS,HOME_Bat Avg,HOME_100s,HOME_Wkts,HOME_BBI,HOME_Bowl Avg,HOME_5w,HOME_Ct,HOME_St,HOME_Avg Diff,AWAY_Span,AWAY_Mat,AWAY_Runs,AWAY_HS,AWAY_Bat Avg,AWAY_100s,AWAY_Wkts,AWAY_BBI,AWAY_Bowl Avg,AWAY_5w,AWAY_Ct,AWAY_St,AWAY_Avg Diff,NEUTRAL_Span,NEUTRAL_Mat,NEUTRAL_Runs,NEUTRAL_HS,NEUTRAL_Bat Avg,NEUTRAL_100s,NEUTRAL_Wkts,NEUTRAL_BBI,NEUTRAL_Bowl Avg,NEUTRAL_5w,NEUTRAL_Ct,NEUTRAL_St,NEUTRAL_Avg Diff,Major trophies,image,career_span,Player Name,Nationality,player-type,BattingMat Test,BattingMat ODI,BattingMat T20I,BattingMat FC,BattingMat List A,BattingMat T20,BattingInns Test,BattingInns ODI,BattingInns T20I,BattingInns FC,BattingInns List A,BattingInns T20,BattingBalls Test,BattingBalls ODI,BattingBalls T20I,BattingBalls FC,BattingBalls List A,BattingBalls T20,BattingRuns Test,BattingRuns ODI,BattingRuns T20I,BattingRuns FC,BattingRuns List A,BattingRuns T20,BattingWkts Test,BattingWkts ODI,BattingWkts T20I,BattingWkts FC,BattingWkts List A,BattingWkts T20,BattingBBI Test,BattingBBI ODI,BattingBBI T20I,BattingBBI FC,BattingBBI List A,BattingBBI T20,BattingBBM Test,BattingBBM ODI,BattingBBM T20I,BattingBBM FC,BattingBBM List A,BattingBBM T20,BattingAve Test,BattingAve ODI,BattingAve T20I,BattingAve FC,BattingAve List A,BattingAve T20,BattingEcon Test,BattingEcon ODI,BattingEcon T20I,BattingEcon FC,BattingEcon List A,BattingEcon T20,BattingSR Test,BattingSR ODI,BattingSR T20I,BattingSR FC,BattingSR List A,BattingSR T20,Batting4w Test,Batting4w ODI,Batting4w T20I,Batting4w FC,Batting4w List A,Batting4w T20,Batting5w Test,Batting5w ODI,Batting5w T20I,Batting5w FC,Batting5w List A,Batting5w T20,Batting10w Test,Batting10w ODI,Batting10w T20I,Batting10w FC,Batting10w List A,Batting10w T20,BowlingMat Test,BowlingMat ODI,BowlingMat T20I,BowlingMat FC,BowlingMat List A,BowlingMat T20,BowlingInns Test,BowlingInns ODI,BowlingInns T20I,BowlingInns FC,BowlingInns List A,BowlingInns T20,BowlingNO Test,BowlingNO ODI,BowlingNO T20I,BowlingNO FC,BowlingNO List A,BowlingNO T20,BowlingRuns Test,BowlingRuns ODI,BowlingRuns T20I,BowlingRuns FC,BowlingRuns List A,BowlingRuns T20,BowlingHS Test,BowlingHS ODI,BowlingHS T20I,BowlingHS FC,BowlingHS List A,BowlingHS T20,BowlingAve Test,BowlingAve ODI,BowlingAve T20I,BowlingAve FC,BowlingAve List A,BowlingAve T20,BowlingBF Test,BowlingBF ODI,BowlingBF T20I,BowlingBF FC,BowlingBF List A,BowlingBF T20,BowlingSR Test,BowlingSR ODI,BowlingSR T20I,BowlingSR FC,BowlingSR List A,BowlingSR T20,Bowling100s Test,Bowling100s ODI,Bowling100s T20I,Bowling100s FC,Bowling100s List A,Bowling100s T20,Bowling50s Test,Bowling50s ODI,Bowling50s T20I,Bowling50s FC,Bowling50s List A,Bowling50s T20,Bowling4s Test,Bowling4s ODI,Bowling4s T20I,Bowling4s FC,Bowling4s List A,Bowling4s T20,Bowling6s Test,Bowling6s ODI,Bowling6s T20I,Bowling6s FC,Bowling6s List A,Bowling6s T20,BowlingCt Test,BowlingCt ODI,BowlingCt T20I,BowlingCt FC,BowlingCt List A,BowlingCt T20,BowlingSt Test,BowlingSt ODI,BowlingSt T20I,BowlingSt FC,BowlingSt List A,BowlingSt T20
0,Mohammed Shami Ahmed,"September 03, 1990, Amroha, Uttar Pradesh",30y 267d,Right hand bat,Right arm fast,Bowler,"[[Mohammed Kaif, (brother)]]","[10th Best career strike rate (27.3), 1st Most...",[16th Most runs in an innings (by batting posi...,[14th Best figures in a innings when on the lo...,[30th Most consecutive matches missed for a te...,"[Test Matches, Debut West Indies vs India at K...","[India, Bengal, Delhi Daredevils, ICC World XI...",2013-2020,44,171,24,14.25,0,109,5/35,25.20,2,9,0,-10.95,2013-2020,78,439,51*,9.34,0,195,6/56,28.97,4,23,0,-19.63,2013-2019,19,49,25,7.00,0,36,4/16,21.91,0,7,0,-14.91,NaN,"https://img1.hscicdn.com/image/upload/f_auto,t...",2013 - 2020,Mohammed Shami,India,Bowler,50,95,8999,4966,180,6/56,9/118,27.58,3.31,49.90,9,5,0,79,78,4044,3793,148,5/69,5/69,25.62,5.62,27.30,9,1,0,12,12,262,428,12,3/38,3

In [46]:
df = a.T
for i in range(1,len(final_list_players)):
    lo = final_list_players[i].T
    df = pd.concat([df,lo],axis=1)

In [47]:
df.T.shape

(10, 323)

In [48]:
df.T

,Full Name,Born,Age,Batting Style,Bowling Style,Playing Role,Relations,Records,Test Records,ODI Records,T20I Records,Matches,Teams,HOME_Span,HOME_Mat,HOME_Runs,HOME_HS,HOME_Bat Avg,HOME_100s,HOME_Wkts,HOME_BBI,HOME_Bowl Avg,HOME_5w,HOME_Ct,HOME_St,HOME_Avg Diff,AWAY_Span,AWAY_Mat,AWAY_Runs,AWAY_HS,AWAY_Bat Avg,AWAY_100s,AWAY_Wkts,AWAY_BBI,AWAY_Bowl Avg,AWAY_5w,AWAY_Ct,AWAY_St,AWAY_Avg Diff,NEUTRAL_Span,NEUTRAL_Mat,NEUTRAL_Runs,NEUTRAL_HS,NEUTRAL_Bat Avg,NEUTRAL_100s,NEUTRAL_Wkts,NEUTRAL_BBI,NEUTRAL_Bowl Avg,NEUTRAL_5w,NEUTRAL_Ct,NEUTRAL_St,NEUTRAL_Avg Diff,Major trophies,image,career_span,Player Name,Nationality,player-type,BattingMat Test,BattingMat ODI,BattingMat T20I,BattingMat FC,BattingMat List A,BattingMat T20,BattingInns Test,BattingInns ODI,BattingInns T20I,BattingInns FC,BattingInns List A,BattingInns T20,BattingBalls Test,BattingBalls ODI,BattingBalls T20I,BattingBalls FC,BattingBalls List A,BattingBalls T20,BattingRuns Test,BattingRuns ODI,BattingRuns T20I,BattingRuns FC,BattingRuns List A,BattingRuns T20,BattingWkts Test,BattingWkts ODI,BattingWkts T20I,BattingWkts FC,BattingWkts List A,BattingWkts T20,BattingBBI Test,BattingBBI ODI,BattingBBI T20I,BattingBBI FC,BattingBBI List A,BattingBBI T20,BattingBBM Test,BattingBBM ODI,BattingBBM T20I,BattingBBM FC,BattingBBM List A,BattingBBM T20,BattingAve Test,BattingAve ODI,BattingAve T20I,BattingAve FC,BattingAve List A,BattingAve T20,BattingEcon Test,BattingEcon ODI,BattingEcon T20I,BattingEcon FC,BattingEcon List A,BattingEcon T20,BattingSR Test,BattingSR ODI,BattingSR T20I,BattingSR FC,BattingSR List A,BattingSR T20,Batting4w Test,Batting4w ODI,Batting4w T20I,Batting4w FC,Batting4w List A,Batting4w T20,Batting5w Test,Batting5w ODI,Batting5w T20I,Batting5w FC,Batting5w List A,Batting5w T20,Batting10w Test,Batting10w ODI,Batting10w T20I,Batting10w FC,Batting10w List A,Batting10w T20,BowlingMat Test,BowlingMat ODI,BowlingMat T20I,BowlingMat FC,BowlingMat List A,BowlingMat T20,BowlingInns Test,BowlingInns ODI,BowlingInns T20I,BowlingInns FC,BowlingInns List A,BowlingInns T20,BowlingNO Test,BowlingNO ODI,BowlingNO T20I,BowlingNO FC,BowlingNO List A,BowlingNO T20,BowlingRuns Test,BowlingRuns ODI,BowlingRuns T20I,BowlingRuns FC,BowlingRuns List A,BowlingRuns T20,BowlingHS Test,BowlingHS ODI,BowlingHS T20I,BowlingHS FC,BowlingHS List A,BowlingHS T20,BowlingAve Test,BowlingAve ODI,BowlingAve T20I,BowlingAve FC,BowlingAve List A,BowlingAve T20,BowlingBF Test,BowlingBF ODI,BowlingBF T20I,BowlingBF FC,BowlingBF List A,BowlingBF T20,BowlingSR Test,BowlingSR ODI,BowlingSR T20I,BowlingSR FC,BowlingSR List A,BowlingSR T20,Bowling100s Test,Bowling100s ODI,Bowling100s T20I,Bowling100s FC,Bowling100s List A,Bowling100s T20,Bowling50s Test,Bowling50s ODI,Bowling50s T20I,Bowling50s FC,Bowling50s List A,Bowling50s T20,Bowling4s Test,Bowling4s ODI,Bowling4s T20I,Bowling4s FC,Bowling4s List A,Bowling4s T20,Bowling6s Test,Bowling6s ODI,Bowling6s T20I,Bowling6s FC,Bowling6s List A,Bowling6s T20,BowlingCt Test,BowlingCt ODI,BowlingCt T20I,BowlingCt FC,BowlingCt List A,BowlingCt T20,BowlingSt Test,BowlingSt ODI,BowlingSt T20I,BowlingSt FC,BowlingSt List A,BowlingSt T20,BattingNO Test,BattingNO ODI,BattingNO T20I,BattingNO FC,BattingNO List A,BattingNO T20,BattingHS Test,BattingHS ODI,BattingHS T20I,BattingHS FC,BattingHS List A,BattingHS T20,BattingBF Test,BattingBF ODI,BattingBF T20I,BattingBF FC,BattingBF List A,BattingBF T20,Batting100s Test,Batting100s ODI,Batting100s T20I,Batting100s FC,Batting100s List A,Batting100s T20,Batting50s Test,Batting50s ODI,Batting50s T20I,Batting50s FC,Batting50s List A,Batting50s T20,Batting4s Test,Batting4s ODI,Batting4s T20I,Batting4s FC,Batting4s List A,Batting4s T20,Batting6s Test,Batting6s ODI,Batting6s T20I,Batting6s FC,Batting6s List A,Batting6s T20,BattingCt Test,BattingCt ODI,BattingCt T20I,BattingCt FC,BattingCt List A,BattingCt T20,BattingSt Test,BattingSt ODI,BattingSt T20I,BattingSt FC,BattingSt List A,BattingSt T20,BowlingBalls Test,BowlingBalls O